# Experimentación - LSTM 



## 1. Obtención de datos



In [32]:

import torch
import torch.nn.functional as F
import pandas as pd
import seaborn as sns
import networkx as nx
sns.set_palette("coolwarm_r")
import matplotlib.pyplot as plt
import numpy as np

from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GCNConv

import os, sys

path = os.getcwd()

sys.path.insert(1, "/".join(path.split("/")[0:-1]))

from utils import powergrid


from utils import pygt_loader

try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable
import os
import torch
import torch.nn.functional as F
from utils.utils_graph import format_plot, plot_training_and_eval_losses, plot_predictions

from importlib import reload  # Python 3.4+
utils.trainer = reload(utils.trainer)
utils.models = reload(utils.models)
from utils.models import LSTMModel
from utils.trainer import  TrainerLSTMModel



dtype = torch.float
device = torch.device("cpu")

In [25]:
#folder_path = "/home/TFM/code_dataset2/datos/Natural Oscillation"
folder_path = "/Users/maguado/Documents/UGR/Master/TFM/datos_2/Natural Oscillation"

In [26]:
loader = powergrid.PowerGridDatasetLoader(folder_path, problem="classification_type")
_,_,_ =loader.process()
limit = 300
dataset_full, situations_each = loader.get_dataset()
target_names = loader.target_names

Processing:  row_307
Node:  0  not included, including...
Node:  1  not included, including...
Node:  2  not included, including...
Node:  3  not included, including...
Node:  4  not included, including...
Node:  5  not included, including...
Node:  6  not included, including...
Node:  7  not included, including...
Node:  8  not included, including...
Node:  9  not included, including...
Node:  10  not included, including...
Node:  11  not included, including...
Node:  12  not included, including...
Node:  13  not included, including...
Node:  14  not included, including...
Node:  15  not included, including...
Node:  16  not included, including...
Node:  17  not included, including...
Node:  18  not included, including...
Node:  19  not included, including...
Node:  20  not included, including...
Node:  21  not included, including...
Node:  22  not included, including...
Processing:  row_135
Processing:  row_75
Processing:  row_338
Processing:  row_81
Processing:  row_132
Processing: 

## Funciones auxiliares - entrenamiento

## Prueba - predicción por tipos - usando batch 

Algunos modelos de los que veremos luego admiten entrenamiento por batch, y otros no, por lo que guardaremos los resultados de LSTM en ambos casos, para poder compararlos.

En GNNs el entrenamiento por batch muchas veces carece de sentido, porque estaríamos modelando todos los grafos del mismo batch como el mismo grafo, perdiendo información que puede ser valiosa.

## Bucle rápido para guardar resultados con batch

In [33]:
dataloader_params2 = {
    "batch_size": 2,
    "data_split_ratio": [0.7, 0.15, 0.15],
    "seed": 42,
    "keep_same": True,
    "use_batch": True
}
params_model = {
    "hidden_size": 50,
    "n_layers": 2
}
problem ="Classification"

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

n_div_prob = loader.div
n_nodes =dataset_full.features[0].shape[0]
n_target = dataset_full.targets[0].shape[0]
n_features = dataset_full[0].x.shape[1]

n_layers = params_model['n_layers']
hidden_size = params_model['hidden_size']


model_prob = LSTMModel(name="LSTM", node_features=n_features, node_count=n_nodes, n_target=n_target, hidden_size=hidden_size, num_layers=n_layers, is_classification=True)

trainer_lstm = TrainerLSTMModel(model_prob, dataset_full,device, f"./results/{problem}", dataloader_params2, batch=True, is_classification=True)
losses, eval_losses, accs, precisions, recalls = trainer_lstm.train(num_epochs=100, steps=100, num_early_stop=15)
test_acc, test_precision, test_recall, test_f1, test_loss, preds, real = trainer_lstm.test(target_names=target_names)






==================== DATASET INFO ===================

Train dataset: 390
Validation dataset: 75
Test dataset: 84

==================== DATALOADER INFO ===================

DataLoader for train set:
Number of batches: 195
DataLoader for val set:
Number of batches: 37
DataLoader for test set:
Number of batches: 42

==================== TRAIN INFO ===================

Epoch 1/100 | Train Loss: 1.5145 | Eval Loss: 1.3496 | Accuracy: 0.5676 | Precision: 0.2297 | Recall: 0.3857 | F1-Score: 0.2877 | LR: 0.0010 | 
Epoch 2/100 | Train Loss: 1.4278 | Eval Loss: 1.3014 | Accuracy: 0.6351 | Precision: 0.4428 | Recall: 0.5000 | F1-Score: 0.4354 | LR: 0.0010 | 
Epoch 3/100 | Train Loss: 1.3357 | Eval Loss: 1.2248 | Accuracy: 0.6892 | Precision: 0.4566 | Recall: 0.6000 | F1-Score: 0.5125 | LR: 0.0010 | 
Epoch 4/100 | Train Loss: 1.2983 | Eval Loss: 1.2369 | Accuracy: 0.6757 | Precision: 0.4500 | Recall: 0.5857 | F1-Score: 0.5033 | LR: 0.0010 | 
Epoch 5/100 | Train Loss: 1.2794 | Eval Loss: 1.2198 |